In [1]:
!pip install accelerate peft bitsandbytes trl
!pip install -U datasets
!pip -q install git+https://github.com/huggingface/transformers.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 8.0 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 11.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 7.2 MB/s eta 0:00:00


In [2]:
!pip install datasets

In [3]:
import torch
import os

from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from transformers import pipeline
from transformers import BitsAndBytesConfig
from transformers import TrainingArguments
from transformers import logging

In [4]:
from peft import LoraConfig
from trl import SFTTrainer

from sklearn.model_selection import train_test_split
from datasets import Dataset ,DatasetDict

In [6]:
from datasets import load_dataset,Dataset

ds = load_dataset("mynkchaudhry/Marketing_Email_dataset")

ds

DatasetDict({
    train: Dataset({
        features: ['product', 'description', 'marketing_email'],
        num_rows: 100
    })
})

In [17]:
from datasets import load_dataset

# Load your dataset
ds = load_dataset("mynkchaudhry/Marketing_Email_dataset")

# Function to combine product and marketing email
def combine_fields(example):
    example['text'] = f"###Product: {example['product']}\n\n###Marketing Email: {example['marketing_email']}"
    return example

# Apply the function to the entire dataset
ds = ds.map(combine_fields)

# Check the result
print(ds['train'][0]['text'])  # Display the combined 'text' field for the first record


###Product: Glow-in-the-dark hiking shoes for night adventures

###Marketing Email: Subject: Light Up Your Night Adventures with Glow-in-the-Dark Hiking Shoes! 🌟

Hey Adventurer,

Are you ready to take your nighttime hikes to a whole new level of excitement? Introducing our latest innovation - Glow-in-the-Dark Hiking Shoes! 🌌

With these luminous kicks, you can light up the trails and explore the great outdoors even after the sun goes down. Made with high-quality materials and designed for both comfort and style, these shoes are perfect for all your nocturnal adventures.

Don't let the darkness limit your exploration - step into a world of endless possibilities with our Glow-in-the-Dark Hiking Shoes!

Shop now and be the shining star of the night! 🌟

Happy exploring,
[Your Name] Marketing Team


In [15]:
ds['train'][0]

{'product': 'Glow-in-the-dark hiking shoes for night adventures',
 'description': '',
 'marketing_email': "Subject: Light Up Your Night Adventures with Glow-in-the-Dark Hiking Shoes! 🌟\n\nHey Adventurer,\n\nAre you ready to take your nighttime hikes to a whole new level of excitement? Introducing our latest innovation - Glow-in-the-Dark Hiking Shoes! 🌌\n\nWith these luminous kicks, you can light up the trails and explore the great outdoors even after the sun goes down. Made with high-quality materials and designed for both comfort and style, these shoes are perfect for all your nocturnal adventures.\n\nDon't let the darkness limit your exploration - step into a world of endless possibilities with our Glow-in-the-Dark Hiking Shoes!\n\nShop now and be the shining star of the night! 🌟\n\nHappy exploring,\n[Your Name] Marketing Team",
 'text': "###Product: Glow-in-the-dark hiking shoes for night adventures\n\n###Marketing Email: Subject: Light Up Your Night Adventures with Glow-in-the-Dark

In [19]:
lora_config=LoraConfig(r=8,target_modules=["q_proj","o_proj","k_proj","v_proj","gate_proj","up_proj","up_proj"],
                                            task_type="CAUSAL_LM",)
                                            
bnb_config=BitsAndBytesConfig(
load_in_4bit=True,bnb_4bit_quant_type="nf4",
bnb_4bit_compute_dtype=torch.bfloat16)

In [20]:
modelpath='/kaggle/input/mistral/pytorch/7b-v0.1-hf/1'
tokenizer=AutoTokenizer.from_pretrained(modelpath)
model=AutoModelForCausalLM.from_pretrained(modelpath,quantization_config=bnb_config,device_map={"":0})

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [22]:
tokenizer.pad_token = tokenizer.eos_token

In [41]:
training_params=TrainingArguments(
    output_dir="./results",
    num_train_epochs=8,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    gradient_checkpointing=True,
    optim="adamw_bnb_8bit",
    save_steps=25,
    logging_steps=100,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.3,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard")
    

In [42]:
trainer = SFTTrainer(
    model=model,
    train_dataset=ds['train'],  # Corrected to use ds['train']
    peft_config=lora_config,
    dataset_text_field="text",  # Ensure this is the correct text field
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:407: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [44]:
torch.cuda.empty_cache()

In [45]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
100,0.829000
200,0.500900
300,0.294800
400,0.201400
500,0.155100
600,0.120400
700,0.101800
800,0.083900


/opt/conda/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/opt/conda/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an except

TrainOutput(global_step=800, training_loss=0.2859340941905975, metrics={'train_runtime': 2484.072, 'train_samples_per_second': 0.322, 'train_steps_per_second': 0.322, 'total_flos': 7618727037960192.0, 'train_loss': 0.2859340941905975, 'epoch': 8.0})

In [46]:

model_save_path = "./mew_model"


trainer.model.save_pretrained(model_save_path)
trainer.tokenizer.save_pretrained(model_save_path)


('./mew_model/tokenizer_config.json',
 './mew_model/special_tokens_map.json',
 './mew_model/tokenizer.model',
 './mew_model/added_tokens.json',
 './mew_model/tokenizer.json')

In [47]:
tokenizer=AutoTokenizer.from_pretrained(model_save_path)


model=AutoModelForCausalLM.from_pretrained(model_save_path,quantization_config=bnb_config,device_map={"":0})

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [48]:
ds['train'][0]['text']

"###Product: Glow-in-the-dark hiking shoes for night adventures\n\n###Marketing Email: Subject: Light Up Your Night Adventures with Glow-in-the-Dark Hiking Shoes! 🌟\n\nHey Adventurer,\n\nAre you ready to take your nighttime hikes to a whole new level of excitement? Introducing our latest innovation - Glow-in-the-Dark Hiking Shoes! 🌌\n\nWith these luminous kicks, you can light up the trails and explore the great outdoors even after the sun goes down. Made with high-quality materials and designed for both comfort and style, these shoes are perfect for all your nocturnal adventures.\n\nDon't let the darkness limit your exploration - step into a world of endless possibilities with our Glow-in-the-Dark Hiking Shoes!\n\nShop now and be the shining star of the night! 🌟\n\nHappy exploring,\n[Your Name] Marketing Team"

In [49]:
text='Glow-in-the-dark hiking shoes for night adventures'
device="cuda:0"
input=tokenizer(text,return_tensors='pt').to(device)
output=model.generate(**input,max_new_tokens=256)
print(tokenizer.decode(output[0],skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Glow-in-the-dark hiking shoes for night adventures!

These shoes are designed for both comfort and style, making them perfect for all your nighttime adventures. With a unique glow-in-the-dark material, these shoes light up the trails and keep you safe and visible in the dark.

Don't let the night limit your exploration - grab a pair of these luminous hiking shoes today and light up the night!

Features:

- Glow-in-the-dark material for visibility in the dark
- Comfortable and stylish design
- Perfect for nighttime hikes, runs, and outdoor adventures
- Keep you safe and seen in low-light conditions
- Made with high-quality materials for durability and comfort

Don't let the sun go down on your adventures - grab a pair of these luminous hiking shoes today and light up the night!

Happy exploring,
[Your Name]
Marketing Executive at [Your Company]

P.S. These shoes are not just for hiking - they're perfect for all your nocturnal adventures, from running to camping to stargazing!

P.P.S. Do

In [21]:
text='Glow-in-the-dark hiking shoes for night adventures'

device='cuda:0'

inputs=tokenizer(text,return_tensors='pt').to(device)

out=model.generate(**inputs,max_new_tokens=250)
print(tokenizer.decode(out[0],skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Glow-in-the-dark hiking shoes for night adventures.

The new Adidas Terrex Fast R GTX hiking shoes are designed for night adventures. The shoes are equipped with a special glow-in-the-dark material that makes them visible in the dark.

The shoes are made of a lightweight and breathable mesh upper, with a waterproof Gore-Tex membrane to keep your feet dry in wet conditions. The midsole is made of a lightweight and cushioned EVA foam, providing comfort and support on long hikes. The outsole is made of a durable and grippy rubber, with deep lugs for traction on various terrain.

The shoes also feature a lace-up closure for a secure fit, and a padded tongue and collar for comfort. The heel counter provides stability and support, and the lace-up closure allows for a customized fit.

The shoes are available in a range of sizes and colors, and are suitable for a variety of outdoor activities, including hiking, trail running, and backpacking. They are also suitable for use in low-light conditi